In [1]:
import caml_core as core
import caml_cell as cell

import cv2
import os
import threading
import yaml
import numpy as np

from pynput.keyboard import Key, Listener

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[WARN    ]	 [2024-04-25 13:19:59.614133] Cannot import pypylon. Will not be able to connect to real Pylon devices 


pybullet build time: May 20 2022 19:44:17


In [2]:
real_sense = cell.sensors.scanners.RealSenseD405(
    serial_number="130322272961",
    frame_rate=15,
)

In [3]:
class Streamer:

    def __init__(self, real_sense: cell.sensors.scanners.RealSenseD405) -> None:
        self._terminate = False
        self._last_observation = None
        self.real_sense = real_sense
        self._trigger_renderer = threading.Event()
        self._camera_thread = None

    def _callback(self, snapshot: core.data.SnapShot) -> bool:
        self._last_observation = snapshot.get_observations()[0]
        self._trigger_renderer.set()
        return not self._terminate

    def _stream(self,) -> None:
        self._camera_thread = threading.Thread(
            target=lambda: self.real_sense.stream(callback=self._callback),
            name="camera-capture",
        )
        self._trigger_renderer.clear()
        self._camera_thread.start()

        while True:
            self._trigger_renderer.wait()
            if not self._terminate:
                self._trigger_renderer.clear()
                h, w, c = self._last_observation.image.get_dimensions()
                resized_image = self._last_observation.image.resize((2 * h, 2 * w))
                cv2.imshow("realsense", resized_image.image[:, :, ::-1])
                cv2.waitKey(10)
            
            else:
                core.logger.info("Closing streaming...")
                cv2.destroyAllWindows()
                self._camera_thread.join()
                core.logger.info("streaming closed")
                break

    def start_stream(self) -> None:
        self._streamer_thread = threading.Thread(target=self._stream, name="streamer")
        self._streamer_thread.start()
        core.logger.info("Streaming started...")

    def terminate(self,) -> None:
        self._terminate = True
        self._trigger_renderer.set()
        self._streamer_thread.join()
        self._terminate = False

In [7]:
class DataCollector:

    def __init__(self, save_dir: str, index: int, config: str, bolt_state: int, hinge_state: int) -> None:
        self.save_dir = save_dir
        if not os.path.exists(self.save_dir):
                os.makedirs(self.save_dir)

        self.index = index
        self.config = config
        # Label details
        self.bolt_state = bolt_state
        self.hinge_state = hinge_state

    def store_observation(self) -> None:
        observation = real_sense.scan().get_observations()[0]

        # flag = input("Use observation? (Y/N)") in ["y", "Y"]
        flag = True

        if flag:
            observation_path = os.path.join(self.save_dir, f"{self.config}_{self.index}")
            core.io.save_observation(observation_path, observation)
            yaml.dump(
                    data={"bolt": self.bolt_state, "hinge": self.hinge_state},
                stream=open(os.path.join(observation_path, "label.yaml"), 'w')
            )
            core.logger.info(f"Sample {self.index} stored.")
            self.index += 1
        
        else:
            core.logger.info("Sample skipped...")



In [8]:
streamer = Streamer(real_sense=real_sense)
data_collector = DataCollector(
    save_dir="data/v2",
    index=4,
    config="bracket_hinge_bolt_nut",
    bolt_state=0,
    hinge_state=0,
)

In [9]:
def on_press(key):
    try:
        print(key.char)
        # Press c to capture
        if key.char == 'c':
            streamer.terminate()
            data_collector.store_observation()
            streamer.start_stream()
            
    except AttributeError:
        if key == Key.esc:
            streamer.terminate()
            return False  # Stop listener

# Start streaming and data-collection
streamer.start_stream()
with Listener(on_press=on_press) as listener:
        listener.join()

[INFO    ]	 [2024-04-25 13:21:14.021596] Streaming started... 
c
[INFO    ]	 [2024-04-25 13:21:16.994164] Closing streaming... 
[INFO    ]	 [2024-04-25 13:21:17.028827] streaming closed 
[INFO    ]	 [2024-04-25 13:21:22.262667] Streaming started... 
n
[INFO    ]	 [2024-04-25 13:21:28.642221] Closing streaming... 
[INFO    ]	 [2024-04-25 13:21:28.694962] streaming closed 
